In [1]:
'''
Manual rebasing script

Purpose: this script takes unrebased projection output and manually calculates the 'combined, rebased impact'
(correctly!) by enacting this equation:

combined_rebased_impact = minlost_hi_rebased * riskshare_hi + minlost_lo_rebased * (1 - riskshare_hi)

This is done for histclim and fulladapt scenarios. As per usual, the histclim value will then be subtracted
from the fulladapt value and we will get our final result.

Parameters:

@model     : the name of the model (and thus the name of folder that output is stored in)
@scenario  : the name of the scenario from which projection output is sourced

Outputs: a modified version of the original projection file that now includes a new column, 'rebased_new',
which has this correctly-calculated combined, rebased impact.

'''
__author__ = 'Kit Schwarz'
__contact__ = 'csschwarz@uchicago.edu'
__version__ = '1.0'

############
# LIBRARIES
############

import xarray as xr
import pandas as pd
import getpass
import sys

/home/nsharma/miniconda3/envs/risingverse/lib/python3.7/site-packages/xarray/core/merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
regions = ['SDN.4.11.49.163', 'THA.3.R3edeff05b7928bfc', 'USA.5.221', 'CAN.3.50.1276', 'CAN.2.33.913', 'GBR.1.R7074136591e79d11'] 

region_dict = {
    'SDN.4.11.49.163' : 'Khartoum, SDN',
    'THA.3.R3edeff05b7928bfc' : 'Bangkok, THA', 
    'USA.5.221' : 'San Francisco, USA',
    'CAN.3.50.1276' : 'Winnipeg, CAN', 
    'CAN.2.33.913' : 'Vancouver, CAN', 
    'GBR.1.R7074136591e79d11' : 'London, GBR'
}

In [3]:
############
# PARAMETERS
############

model = 'labor-climtasmaxclip'
# select: uninteracted_main_model_w_chn, uninteracted_main_model

############
# PATHWAYS
############

username = getpass.getuser()

if model == 'uninteracted_main_model_w_chn':
    
    proj_root = ('/shares/gcp/outputs/labor/impacts-woodwork/uninteracted_main_model_w_chn/' +
                 'uninteracted_splines_w_chn_21_37_41_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3')
    
elif model == 'uninteracted_main_model':
    
    proj_root = ('/shares/gcp/outputs/labor/impacts-woodwork/test_rcc_copy1/' +
                 'uninteracted_splines_27_37_39_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3')

elif model == 'labor-climtasmaxclip':
    
    proj_root = ('/shares/gcp/outputs/labor/impacts-woodwork/main_model_flat_edges_single_copy/' +
                 'uninteracted_splines_27_37_39_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3')

else:

    sys.exit("Your model is unrecognized.")
    
############
# FUNCTION
############

def rebase_df(proj_root, model, scenario):
    dt = (xr.open_dataset(
            f'{proj_root}/{model}{scenario}.nc4')
            .to_dataframe()
            .reset_index()
         )

    # to get rid of duplicate rows (because of orderofoperations column)
    dt = dt.loc[dt.orderofoperations == 'clip'] 

    # get a sub-dataset for the base years used to calculate rebasing
    base = dt.loc[(dt.year >=2001) & (dt.year <=2010)]

    # get mean by region for the base years
    mean_base = base.groupby("regions").agg(
                        high_base=('highriskimpacts', 'mean'),
                         low_base=('lowriskimpacts', 'mean'))

    # merge base year values onto main dataset so we can calculate new columns
    dt = dt.merge(mean_base,
                left_on='regions',
                right_index = True)

    # these are the rebased sector-specific impacts
    dt['rebased_high'] = dt.highriskimpacts - dt.high_base
    dt['rebased_low'] = dt.lowriskimpacts - dt.low_base

    # and the final weighted average (combined, rebased) impacts
    # this is how manual_rebasing_MC.py and manual_rebasing_single.py calculate rebased_new
    dt['rebased_new'] = (dt.rebased_high * dt['clip']) + (dt.rebased_low * (1 - dt['clip']))
    
    return dt

In [4]:
########################
# IMPLEMENT MANUAL CHECK
########################

fulladapt = rebase_df(proj_root, model, '').set_index(['regions', 'year'])
histclim = rebase_df(proj_root, model, '-histclim').set_index(['regions', 'year'])

results = pd.DataFrame(index=fulladapt.index)

# these are rebased total impacts
results['impact_H'] = (
            (fulladapt['highriskimpacts'] - fulladapt['high_base'])
            - (histclim['highriskimpacts'] - histclim['high_base'])
            )

results['impact_L'] = (
            (fulladapt['lowriskimpacts'] - fulladapt['low_base'])
            - (histclim['lowriskimpacts'] - histclim['low_base'])
            )

# these are the calculation of terms found in Ashwin's memo on the 
# projection system mess-up
results['weighted_term'] = (
                (fulladapt['clip'] * results['impact_H'])
                + ((1- fulladapt['clip']) * results['impact_L'])
                )

results['extra_term'] = (
        (fulladapt['clip'] - histclim['clip']) 
        * ((histclim['highriskimpacts'] - histclim['high_base']) 
        - (histclim['lowriskimpacts'] - histclim['low_base']))
        )

# this is the impact calculated from the rearranged equation
results['impact_summed_terms'] = results['weighted_term'] + results['extra_term']

# this is the impact as calculated in the manual_rebasing_MC and manual_rebasing_singles scripts
results['impact_manual_rebasing_script'] = fulladapt['rebased_new'] - histclim['rebased_new']

In [5]:
# results for fulladapt - histclim

region_data = results.loc[results.index.isin(regions, level='regions') & results.index.isin([2099], level='year')].reset_index()
region_data['name'] = region_data['regions'].apply(lambda x : region_dict[x])
region_data = region_data.set_index(['name'])
region_data.T

name,"Winnipeg, CAN","Vancouver, CAN","San Francisco, USA","Bangkok, THA","Khartoum, SDN","London, GBR"
regions,CAN.3.50.1276,CAN.2.33.913,USA.5.221,THA.3.R3edeff05b7928bfc,SDN.4.11.49.163,GBR.1.R7074136591e79d11
year,2099,2099,2099,2099,2099,2099
impact_H,2.91639,4.15008,2.68805,-18.7285,-23.1056,1.39792
impact_L,0.0327912,0.216076,0.164934,-3.45856,-4.15378,0.0953954
weighted_term,0.51129,0.398788,0.282118,-6.54819,-11.6347,0.275132
extra_term,-0.0299923,0.0637306,-0,0.00768544,0,-0.0117989
impact_summed_terms,0.481298,0.462518,0.282118,-6.5405,-11.6347,0.263333
impact_manual_rebasing_script,0.481298,0.462518,0.282118,-6.5405,-11.6347,0.263333


In [6]:
# results for fulladapt

regions_fulladapt = fulladapt.loc[fulladapt.index.isin(regions, level='regions') & fulladapt.index.isin([2099], level='year')].reset_index()
regions_fulladapt['name'] = regions_fulladapt['regions'].apply(lambda x : region_dict[x])
regions_fulladapt = regions_fulladapt.set_index(['name'])
regions_fulladapt = regions_fulladapt.drop(columns=['operation', 'region'])
regions_fulladapt = regions_fulladapt.rename(columns={'clip' : 'highrisk_share',
                                             'rebased' : 'wrong_rebasing'})

regions_fulladapt.T

name,"Winnipeg, CAN","Vancouver, CAN","San Francisco, USA","Bangkok, THA","Khartoum, SDN","London, GBR"
regions,CAN.3.50.1276,CAN.2.33.913,USA.5.221,THA.3.R3edeff05b7928bfc,SDN.4.11.49.163,GBR.1.R7074136591e79d11
year,2099,2099,2099,2099,2099,2099
wrong_rebasing,-1.07175,-1.92309,-0.947342,-9.55687,-9.16703,-1.34926
lowriskimpacts,0.49515,0.860154,1.15657,-2.82554,-7.2676,0.825951
highriskimpacts,9.90051,13.5095,17.1006,-0.679784,-25.6499,12.0146
highrisk_share,0.165938,0.0464441,0.0464441,0.202335,0.394735,0.137991
orderofoperations,clip,clip,clip,clip,clip,clip
rebased_new,0.558803,0.313545,0.253245,-5.7632,-9.79138,0.315143
high_base,6.72349,10.1218,14.6719,15.826,-6.196,10.4142
low_base,0.457245,0.696343,1.00928,0.212718,-3.77783,0.716556


In [7]:
# results for histclim

regions_histclim = histclim.loc[histclim.index.isin(regions, level='regions') & histclim.index.isin([2099], level='year')].reset_index()
regions_histclim['name'] = regions_histclim['regions'].apply(lambda x : region_dict[x])
regions_histclim = regions_histclim.set_index(['name'])
regions_histclim = regions_histclim.drop(columns=['operation', 'region'])
regions_histclim = regions_histclim.rename(columns={'clip' : 'highrisk_share',
                                             'rebased' : 'wrong_rebasing'})


regions_histclim.T

name,"Winnipeg, CAN","Vancouver, CAN","San Francisco, USA","Bangkok, THA","Khartoum, SDN","London, GBR"
regions,CAN.3.50.1276,CAN.2.33.913,USA.5.221,THA.3.R3edeff05b7928bfc,SDN.4.11.49.163,GBR.1.R7074136591e79d11
year,2099,2099,2099,2099,2099,2099
wrong_rebasing,-0.827081,-1.56055,-1.25305,-3.21486,2.39308,-1.00141
lowriskimpacts,0.452425,0.638293,0.987663,0.79306,-3.05087,0.706389
highriskimpacts,6.8699,9.27542,14.3525,18.866,-2.1923,10.2627
highrisk_share,0.283319,0.136198,0.0464441,0.198071,0.394735,0.200584
orderofoperations,clip,clip,clip,clip,clip,clip
rebased_new,0.0775054,-0.148973,-0.028873,0.777303,1.84336,0.0518098
high_base,6.60928,10.0377,14.6118,16.6434,-5.84401,10.0602
low_base,0.447311,0.690558,1.00531,0.372756,-3.71488,0.69239


In [8]:
# get indices of the regions we want to print out

idx = results.reset_index().index[results.index.isin(regions, level = 'regions') & results.index.isin([2099], level='year')].to_list()

In [9]:
def print_results(i):
    
    print(f"***************************** \r\n Computing region {results.iloc[i].name} ... \r\n")
    print(
        "results['weighted_term'] = (fulladapt['clip'] * results['impact_H']) + ((1- fulladapt['clip']) * results['impact_L']) \r\n"
    )

    print(f"{results['weighted_term'].iloc[i]} = ({fulladapt['clip'].iloc[i]} * {results['impact_H'].iloc[i]})",
          f"+ ((1- {fulladapt['clip'].iloc[i]}) * {results['impact_L'].iloc[i]}) \r\n"
    )

    print("results['extra_term'] = ((fulladapt['clip'] - histclim['clip']) \r\n",
        " * ((histclim['highriskimpacts'] - histclim['high_base']) - (histclim['lowriskimpacts'] - histclim['low_base']))) \r\n"
         )

    print(f"{results['extra_term'].iloc[i]} = (({fulladapt['clip'].iloc[i]} - {histclim['clip'].iloc[i]}) \r\n",
        f" * (({histclim['highriskimpacts'].iloc[i]} - {histclim['high_base'].iloc[i]}) - ({histclim['lowriskimpacts'].iloc[i]} - {histclim['low_base'].iloc[i]}))) \r\n"
         )

    print("results['impact_summed_terms'] = results['weighted_term'] + results['extra_term'] \r\n")

    print(f"{results['impact_summed_terms'].iloc[i]} = {results['weighted_term'].iloc[i]} + {results['extra_term'].iloc[i]} \r\n")

    return

In [10]:
for i in idx:
    print_results(i)

***************************** 
 Computing region ('CAN.3.50.1276', 2099) ... 

results['weighted_term'] = (fulladapt['clip'] * results['impact_H']) + ((1- fulladapt['clip']) * results['impact_L']) 

0.5112901926040649 = (0.16593794524669647 * 2.916393280029297) + ((1- 0.16593794524669647) * 0.032791197299957275) 

results['extra_term'] = ((fulladapt['clip'] - histclim['clip']) 
  * ((histclim['highriskimpacts'] - histclim['high_base']) - (histclim['lowriskimpacts'] - histclim['low_base']))) 

-0.029992274940013885 = ((0.16593794524669647 - 0.2833193838596344) 
  * ((6.869902610778809 - 6.609277248382568) - (0.4524252712726593 - 0.447311133146286))) 

results['impact_summed_terms'] = results['weighted_term'] + results['extra_term'] 

0.48129791021347046 = 0.5112901926040649 + -0.029992274940013885 

***************************** 
 Computing region ('CAN.2.33.913', 2099) ... 

results['weighted_term'] = (fulladapt['clip'] * results['impact_H']) + ((1- fulladapt['clip']) * results['impact

In [11]:
WI

NameError: name 'WI' is not defined